<a href="https://colab.research.google.com/github/nebojsa55/Computational-Genomics_MidTerm-Project/blob/master/notebooks/1.%20Basic-regression-models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we will look at two regression models (**Random forest regressor** and **SVR**) and their respective accuracies when predicting gestational age in pregnant women using gene expression. In the second notebook, we will try to implement ensemble regressor by utilizing two predefined models and using linear regression to get a linear ensemble regressor.

In [1]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

# Navigate to the folder cointaing our data
%cd 'drive/MyDrive/ETF/Master/Computational-Genomics/Project/data'


Mounted at /content/drive
/content/drive/MyDrive/ETF/Master/Computational-Genomics/Project/data


In [2]:
# Verify the chosen folder
!ls

anoSC1_v11_nokey.csv  anoSC2_v20_nokey.csv  eset_HuGene21ST.csv  HTA20_RMA.csv


In [3]:
anno = pd.read_csv('anoSC1_v11_nokey.csv', delimiter = ',', index_col = 0)
anno.head()

,GA,Batch,Set,Train,Platform
SampleID,,,,,
Tarca_001_P1A01,11.0,1,PRB_HTA,1,HTA20
Tarca_013_P1B01,15.3,1,PRB_HTA,1,HTA20
Tarca_025_P1C01,21.7,1,PRB_HTA,1,HTA20
Tarca_037_P1D01,26.7,1,PRB_HTA,1,HTA20
Tarca_049_P1E01,31.3,1,PRB_HTA,1,HTA20


In [4]:
HTA20_RMA = pd.read_csv('HTA20_RMA.csv', delimiter = ',', index_col = 0).transpose()
HTA20_RMA.head()

,1_at,10_at,100_at,1000_at,10000_at,100009613_at,100009676_at,10001_at,10002_at,10003_at,100033411_at,100033413_at,100033414_at,100033418_at,100033420_at,100033422_at,100033423_at,100033424_at,100033425_at,100033426_at,100033427_at,100033428_at,100033430_at,100033431_at,100033432_at,100033433_at,100033434_at,100033435_at,100033436_at,100033437_at,100033438_at,100033439_at,100033440_at,100033441_at,100033444_at,100033445_at,100033447_at,100033451_at,100033453_at,100033603_at,...,9989_at,999_at,9990_at,9991_at,9992_at,9993_at,9994_at,9995_at,9997_at,AFFX-BkGr-GC03_at,AFFX-BkGr-GC04_at,AFFX-BkGr-GC05_at,AFFX-BkGr-GC06_at,AFFX-BkGr-GC07_at,AFFX-BkGr-GC08_at,AFFX-BkGr-GC09_at,AFFX-BkGr-GC10_at,AFFX-BkGr-GC11_at,AFFX-BkGr-GC12_at,AFFX-BkGr-GC13_at,AFFX-BkGr-GC14_at,AFFX-BkGr-GC15_at,AFFX-BkGr-GC16_at,AFFX-BkGr-GC17_at,AFFX-BkGr-GC18_at,AFFX-BkGr-GC19_at,AFFX-BkGr-GC20_at,AFFX-BkGr-GC21_at,AFFX-BkGr-GC22_at,AFFX-BkGr-GC23_at,AFFX-BkGr-GC24_at,AFFX-BkGr-GC25_at,AFFX-r2-Bs-dap-5_st,AFFX-r2-Bs-lys-5_st,AFFX-r2-Bs-phe-5_st,AFFX-r2-Bs-thr-5_st,AFFX-r2-Ec-bioB-5_at,AFFX-r2-Ec-bioC-5_at,AFFX-r2-Ec-bioD-5_at,AFFX-r2-P1-cre-5_at
Tarca_001_P1A01,6.062215,3.796484,5.849338,3.567779,6.166815,4.443027,5.836522,6.330018,4.922339,2.689344,2.745619,2.760609,3.794684,2.848183,2.812774,2.140590,2.692848,2.352924,2.484768,6.538760,9.544726,2.362534,2.406803,4.657512,3.047816,2.109620,5.771233,4.077138,3.123733,2.195346,2.975659,3.324574,2.430415,2.135031,2.073197,2.400044,2.235433,2.274116,2.261528,2.395431,...,8.815727,5.624144,9.769020,9.700440,3.771009,8.227688,5.992462,5.387680,7.220196,2.191452,2.111011,2.108250,2.120018,2.159772,2.215722,2.223840,2.286487,2.371125,2.499349,2.741896,3.064696,3.792943,4.401289,4.918235,5.631017,5.963709,6.412206,7.216170,7.863468,8.502729,8.786114,11.587051,8.342203,4.520028,5.636615,6.709797,8.972873,10.440245,12.101476,13.695705
Tarca_003_P1A03,6.125023,3.805305,6.191562,3.452524,5.678373,4.773199,6.143398,5.601745,4.711765,2.771010,3.263577,2.402899,3.346500,2.295464,2.752254,2.230644,2.808706,2.744754,2.392213,6.467364,8.983699,2.072872,2.145610,4.924804,2.951409,2.159747,5.714813,3.980073,2.873573,2.296149,2.765574,3.076334,1.960614,2.248927,2.554416,2.177165,2.065589,2.202918,2.091162,1.976425,...,8.226103,6.582291,9.389587,9.271577,3.838563,8.083417,5.826221,5.780958,7.440989,2.011836,2.053886,2.062657,2.106248,2.107264,2.171664,2.195808,2.285022,2.385383,2.570683,2.905709,3.323025,4.155433,4.921309,5.625932,6.350828,6.794233,7.217978,8.070590,8.642121,9.393085,9.594813,12.138090,9.010691,5.148384,6.723139,6.153199,9.376194,10.845176,12.370891,13.635522
Tarca_004_P1A04,5.875502,3.450245,6.550525,3.316134,6.185059,4.393488,5.898364,6.137984,4.628124,2.556756,2.707656,3.036456,4.299712,2.483257,3.513083,2.268800,3.034612,2.385361,2.288809,7.389350,10.039720,2.154480,2.025051,5.993964,2.810200,1.943169,6.154146,4.331164,2.710385,2.238039,2.833525,3.401957,2.044615,2.048938,1.958607,1.918034,1.815077,2.009749,2.109266,2.257260,...,8.454372,5.852143,9.431374,9.470850,3.568739,7.983239,6.175550,5.393079,7.331443,2.088989,2.003418,2.026268,2.039770,2.045315,2.089306,2.105618,2.184611,2.283092,2.430860,2.698857,3.048324,3.767554,4.423481,5.102220,5.843332,6.258922,6.762380,7.623269,8.280635,9.033983,9.294845,12.049271,8.555541,4.441900,6.016953,9.590764,8.843612,10.493416,12.295786,13.616688
Tarca_005_P1A05,6.126131,3.628411,6.421877,3.432451,5.633757,4.623783,6.019792,5.787502,4.796283,2.613415,2.807741,2.645834,3.810852,3.238697,2.784896,2.331521,2.756186,2.362922,2.314093,6.896619,9.346465,2.051147,2.134905,4.942601,3.305603,2.407998,5.454726,4.206752,2.410564,2.268998,2.669970,3.014598,2.033149,2.310311,2.188688,1.991138,2.243578,2.077085,2.033383,2.458295,...,8.606502,6.114320,9.544130,9.374321,3.691639,8.270554,5.635078,5.183500,7.298491,2.044670,2.094643,2.139602,2.094847,2.126440,2.150045,2.199919,2.325623,2.383102,2.567234,2.876098,3.277482,4.066424,4.859661,5.400951,6.173044,6.645898,7.140619,7.980902,8.64

## Missing data

Unfortunately, all of the test data are missing **GA** (gastetion age) parameter, due to the nature od DREAM Pretem Challenge. In order to overcome this, we will use only the original training set for our model training and testing. 10% of data was used for testing (37 samples). 10-fold cross validation was performed in order to determine our ML model.

In [5]:
anno.loc[anno['Train'] == 1].shape[0]

367

In [6]:
# Sync the X and y data by sorting the labels

df1 = anno.sort_index()
df2 = HTA20_RMA.sort_index()

X = df2.iloc[np.array(np.logical_not(df1['GA'].isna())),:]
y = df1.dropna().loc[:,['GA','Batch']]


In [7]:
# Check to see if the indexes are the same
(X.index == y.index).all()

True

## Standard scaling
We will now perform the standard scaling of features from *one batch*

In [8]:
from sklearn.preprocessing import StandardScaler

XX = np.zeros(X.shape)
for i in [1,2,3,4,5,6,7,8,9,10,32]:
    scale = StandardScaler()
    indices = np.bool8(y['Batch'] == i)
    Xtemp = X.iloc[indices,:]
    scale.fit(Xtemp)
    XX[indices,:] = scale.transform(Xtemp)

In [9]:
# delete batch column
yy = y['GA']

## Regression

In this section we will perform regression using a couple of well known regressors such as:
* **Random forest regressor** and
* **Support vector regressor**

The main regression metric will be **RMSE** (Root mean square error), as it was used in DREAM Preterm challenge. 

The total number of features is 32830, which is a large number. We will shrink the number of features via **PCA analysis** to obtain 95% variance. The PCA will give us a total of /// features.

### Random forest regressor

In [10]:
# import necessary ML modules
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from tqdm import tqdm


num_of_splits = 10
kf = KFold(n_splits = num_of_splits)
regressor = RandomForestRegressor(n_estimators = 1000,
                                  n_jobs = -1, 
                                  random_state = 42)

pca = PCA(n_components = 0.95, svd_solver = 'full')

y_train_pred = []
y_test_pred = []

for train_index, test_index in tqdm(kf.split(XX), total = num_of_splits, unit = 'iteration'):

  X_train_std, X_test_std = XX[train_index,:], XX[test_index,:]
  y_train, y_test = yy.iloc[train_index], yy.iloc[test_index]

  pca.fit(XX)
  
  X_train_pca = pca.transform(X_train_std)
  X_test_pca = pca.transform(X_test_std)
  
  regressor.fit(X_train_pca,y_train)
  
  y_train_pred.append(mean_squared_error(y_train,regressor.predict(X_train_pca), squared = False))
  y_test_pred.append(mean_squared_error(y_test,regressor.predict(X_test_pca), squared = False))



100%|██████████| 10/10 [05:39<00:00, 33.96s/iteration]


In [11]:
np.mean(y_train_pred)

2.7622168255170916

In [12]:
np.mean(y_test_pred)

7.532737936379085

#### Model overfitting
We can observe that our current RF model is heavily overfiting. To overcome that, we will try to tune hyperparameters via **Randomized Search crossvalidation**

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV

scorer = make_scorer(mean_squared_error, greater_is_better = False)

n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1000, num = 5)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 100, num = 10)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 4]

# Method of selecting samples for training each tree
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
                }


# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, 
                               param_distributions = random_grid, 
                               n_iter = 20, 
                               cv = 5, 
                               verbose = 3, 
                               random_state = 42)

XX_pca = pca.transform(XX)
rf_random.fit(XX_pca,yy)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=70 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=70, score=-0.023, total=   0.7s
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=70 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s remaining:    0.0s


[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=70, score=0.063, total=   0.6s
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=70 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.3s remaining:    0.0s


[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=70, score=0.057, total=   0.6s
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=70 
[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=70, score=0.082, total=   0.6s
[CV] n_estimators=200, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=70 
[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=70, score=0.059, total=   0.6s
[CV] n_estimators=800, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=20 
[CV]  n_estimators=800, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=20, score=-0.018, total=   2.3s
[CV] n_estimators=800, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=20 
[CV]  n_estimators=800, min_samples_split=10, min_samples_leaf=4, max_features=sqrt, max_depth=20, scor

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  8.4min finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=100,
                              

In [18]:
rf_random.get_params

<bound method BaseEstimator.get_params of RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_esti

In [21]:

n_estimators = [100, 1000, 3000]

# Number of features to consider at every split
max_features = ['auto']

# Maximum number of levels in tree
max_depth = [3, None]

# Minimum number of samples required to split a node
min_samples_split = [2, 4]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1,10]

# Method of selecting samples for training each tree
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
                }


# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_grid = GridSearchCV(estimator = rf, 
                       param_grid = param_grid,  
                       cv = 5, 
                       verbose = 3)

rf_grid.fit(XX_pca,yy)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100, score=0.103, total=   1.5s
[CV] max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.5s remaining:    0.0s


[CV]  max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100, score=0.224, total=   1.5s
[CV] max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.0s remaining:    0.0s


[CV]  max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100, score=0.189, total=   1.5s
[CV] max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100 
[CV]  max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100, score=0.262, total=   1.4s
[CV] max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100 
[CV]  max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100, score=0.084, total=   1.4s
[CV] max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=1000 
[CV]  max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=1000, score=0.124, total=  14.2s
[CV] max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=1000 
[CV]  max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=1000, score=0.251, total=

[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed: 52.4min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0,
                                             criterion='mse', max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             max_samples=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=100, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='deprecated', n_jo

In [24]:
rf_grid.best_params_

{'max_depth': None,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 100}

In [26]:
from sklearn.svm import SVR


svr = GridSearchCV(SVR(kernel='rbf', gamma = 0.1),
                   param_grid={"C": [1e0, 1e1, 1e2, 1e3],
                               "gamma": np.logspace(-2, 2, 5)},
                   scoring = scorer,
                   cv = 5,
                   verbose = 2)

svr.fit(XX,yy)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] C=1.0, gamma=0.01 ...............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................................ C=1.0, gamma=0.01, total=   6.0s
[CV] C=1.0, gamma=0.01 ...............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.0s remaining:    0.0s


[CV] ................................ C=1.0, gamma=0.01, total=   5.9s
[CV] C=1.0, gamma=0.01 ...............................................
[CV] ................................ C=1.0, gamma=0.01, total=   6.0s
[CV] C=1.0, gamma=0.01 ...............................................
[CV] ................................ C=1.0, gamma=0.01, total=   6.0s
[CV] C=1.0, gamma=0.01 ...............................................
[CV] ................................ C=1.0, gamma=0.01, total=   5.9s
[CV] C=1.0, gamma=0.1 ................................................
[CV] ................................. C=1.0, gamma=0.1, total=   5.8s
[CV] C=1.0, gamma=0.1 ................................................
[CV] ................................. C=1.0, gamma=0.1, total=   5.9s
[CV] C=1.0, gamma=0.1 ................................................
[CV] ................................. C=1.0, gamma=0.1, total=   5.9s
[CV] C=1.0, gamma=0.1 ................................................
[CV] .

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  9.9min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma=0.1, kernel='rbf', max_iter=-1,
                           shrinking=True, tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1.0, 10.0, 100.0, 1000.0],
                         'gamma': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=make_scorer(mean_squared_error, greater_is_better=False),
             verbose=2)

In [27]:
svr_best = svr.best_estimator_

pca = PCA(n_components = 0.95, svd_solver = 'full')

y_train_pred = []
y_test_pred = []

for train_index, test_index in tqdm(kf.split(XX), total = num_of_splits, unit = 'iteration'):

  X_train_std, X_test_std = XX[train_index,:], XX[test_index,:]
  y_train, y_test = yy.iloc[train_index], yy.iloc[test_index]

  pca.fit(XX)
  
  X_train_pca = pca.transform(X_train_std)
  X_test_pca = pca.transform(X_test_std)
  
  svr_best.fit(X_train_pca,y_train)
  
  y_train_pred.append(mean_squared_error(y_train,svr_best.predict(X_train_pca), squared = False))
  y_test_pred.append(mean_squared_error(y_test,svr_best.predict(X_test_pca), squared = False))

100%|██████████| 10/10 [00:50<00:00,  5.10s/iteration]


In [28]:
np.mean(y_train_pred)

7.57190670704602

In [29]:
np.mean(y_test_pred)

8.408145494162543